In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [1]:
import os

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='False'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.90'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR']='platform'

import jax
# Global flag to set a specific platform, must be used at startup.
#jax.config.update('jax_platform_name', 'cpu')

import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
import jax
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params

In [2]:
import gc
def clear_jax_caches():
  """Utility to clear all the function caches in jax."""
  # main jit/pmap lu wrapped function caches - have to grab from closures
  jax.xla._xla_callable.__closure__[1].cell_contents.clear()
  jax.pxla.parallel_callable.__closure__[1].cell_contents.clear()
  # primitive callable caches
  jax.xla.xla_primitive_callable.cache_clear()
  jax.xla.primitive_computation.cache_clear()
  # jaxpr caches for control flow and reductions
  jax.lax.lax_control_flow._initial_style_jaxpr.cache_clear()
  jax.lax.lax_control_flow._fori_body_fun.cache_clear()
  jax.lax.lax._reduction_jaxpr.cache_clear()
  # these are trivial and only included for completeness sake
  jax.lax.lax.broadcast_shapes.cache_clear()
  jax.xla.xb.get_backend.cache_clear()
  jax.xla.xb.dtype_to_etype.cache_clear()
  jax.xla.xb.supported_numpy_dtypes.cache_clear()
    
def reset_device_memory(delete_objs=True):
    """Free all tracked DeviceArray memory and delete objects.

  Args:
    delete_objs: bool: whether to delete all live DeviceValues or just free.

  Returns:
    number of DeviceArrays that were manually freed.
  """
    dvals = (x for x in gc.get_objects() if isinstance(x, jax.xla.DeviceArray))
    n_deleted = 0
    for dv in dvals:
    
        if not isinstance(dv, jax.xla.DeviceConstant):
            try: 
                dv._check_if_deleted()  # pylint: disable=protected-access
                dv.device_buffer.delete()
                n_deleted += 1
            except:
                pass
        if delete_objs:

            del dv
    del dvals
    gc.collect()
    return n_deleted


In [3]:
fastas = "/home/ubuntu/data/bk_fasta/ALK-2620-02_S2_L001.assembly.len10.cd100.fasta"
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,34,LVLLNTEERPHFYTLYLEEPDFSGHRYGPVSSEVILALQRVDKVVE...,116
1,38,RKKQYLRLLASTILPLNKYNQKQKGQTFVFGRKYDREFGLGSRRTL...,122
2,59,LSVVDKVTDENILRSANWSPLFNATILWACQINSTSFVSKGICMTL...,106
3,61,AHFENRKKEEEELIFLKDRIEQRRAERAEQQRIRSEREKERQARLA...,101
4,69,KQKVTISGESSGWAASDKRVSQGSVLGPILFNLFINDMEEGVNSLL...,166
...,...,...,...
1468918,34495360,IRKNFLFFIRSGWVDFTIPTFHSQYPEDTGHARRKPFSFNGERLLF...,100
1468919,34495366,WTKQLTKALLSFVDDLPPTRPKRPHALPTKLFIPSGSLTDTLRDII...,138
1468920,34495367,LEYHPPCCPAESLPELMEVVLDSVLRTARKMVLGDLNFCTEATLSG...,119
1468921,34495419,SKTTQRRIVGEALNTQRLAWNHVNYGSISRFEGFWTIFQLLTRTTI...,89


In [4]:
# Setting my chunk size
chunk_size = 2000
# Assigning chunk numbers to rows
df['chunk'] = df['ID'].apply(lambda x: int(int(x)/ chunk_size))
# We don't want the 'chunk' and 'index' columns in the output
cols = [col for col in df.columns if col not in ['chunk']]
# groupby chunk and export each chunk to a different csv.
i = 0
for _, chunk in df.groupby('chunk'):
    chunk[cols].to_csv(f'/mnt/vdb/komodo/cd100/chunks/chunk{i}.csv', index=False)
    i += 1
print("complete")

complete


In [3]:
def createREPs(df, filename):
    _h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
    df.drop(columns=['Sequence'], inplace=True)
    df['reps']=_h_avg.tolist() # if there is a problem , might be here , possible solution is reindexing
    df.to_pickle(filename) # dont forget to change the file name to 0_613834

In [4]:
import glob
appended_reps = []
for infile in glob.glob("/mnt/vdb/komodo/cd100/reps/*.pkl"):
    # print(infile)
    appended_reps.append(infile)
print(len(appended_reps))

626


In [ ]:
for infile in glob.glob("/mnt/vdb/komodo/cd100/chunks/*.csv"):
    #print("Read:"+infile)
    file_name = os.path.basename(infile)
    result="/mnt/vdb/komodo/cd100/reps/"+file_name.replace("csv", "pkl")
    if result in appended_reps :
        # print("found then skip : " , result)
        continue
    else:
        df = pd.read_csv(infile) 
        createREPs(df,result)
        print("Save:"+result)
        reset_device_memory()
        clear_jax_caches()
print("complete")

In [ ]:
df.to_pickle(plk)
